In [4]:
import json 
import requests
 
api_key = ""
filename = "5 고추 최종파일_단면.pdf"
 
url = "https://api.upstage.ai/v1/document-digitization"
headers = {"Authorization": f"Bearer {api_key}"}
files = {"document": open(filename, "rb")}
data = {"ocr": "force", "base64_encoding": "['table']", "model": "document-parse"}
response = requests.post(url, headers=headers, files=files, data=data)

In [5]:
response.json()

{'error': {'message': 'The uploaded document has too many pages. Please split your document into smaller parts or include only the necessary pages.',
  'type': 'invalid_request_error',
  'code': 'invalid_document',
  'param': 'document'}}

In [8]:
! pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.7/309.7 KB 7.8 MB/s eta 0:00:0000:01


In [11]:
from pypdf import PdfReader, PdfWriter
import os
import re

def split_pdf(input_path, output_dir, chunk_size=5):
    reader = PdfReader(input_path)
    total_pages = len(reader.pages)

    file_name = os.path.basename(input_path)
    cleaned_name = re.sub(r"\.pdf$", "", file_name, flags=re.IGNORECASE)

    for i in range(0, total_pages, chunk_size):
        writer = PdfWriter()
        for j in range(i, min(i + chunk_size, total_pages)):
            writer.add_page(reader.pages[j])
        
        output_path = f"{output_dir}/{cleaned_name}_{i+1}_{min(i+chunk_size, total_pages)}.pdf"
        with open(output_path, "wb") as f:
            writer.write(f)
        print(f"✅ Saved: {output_path}")

split_pdf("./raw_pdfs/20 무_고화질_단면.pdf", "./split_pdfs", chunk_size=20)

✅ Saved: ./split_pdfs/20 무_고화질_단면_1_20.pdf
✅ Saved: ./split_pdfs/20 무_고화질_단면_21_40.pdf
✅ Saved: ./split_pdfs/20 무_고화질_단면_41_60.pdf
✅ Saved: ./split_pdfs/20 무_고화질_단면_61_80.pdf
✅ Saved: ./split_pdfs/20 무_고화질_단면_81_100.pdf
✅ Saved: ./split_pdfs/20 무_고화질_단면_101_120.pdf
✅ Saved: ./split_pdfs/20 무_고화질_단면_121_140.pdf
✅ Saved: ./split_pdfs/20 무_고화질_단면_141_156.pdf


In [26]:
import os 
import requests
import json

def split_pdf(input_path, output_dir, chunk_size):
    reader = PdfReader(input_path)
    total_pages = len(reader.pages)
    split_files = []

    file_name = os.path.basename(input_path)
    cleaned_name = re.sub(r"\.pdf$", "", file_name, flags=re.IGNORECASE)

    for i in range(0, total_pages, chunk_size):
        writer = PdfWriter()
        for j in range(i, min(i + chunk_size, total_pages)):
            writer.add_page(reader.pages[j])
        
        split_filename = os.path.join(output_dir, f"{cleaned_name}_{i+1}_{min(i + chunk_size, total_pages)}.pdf")
        with open(split_filename, "wb") as f:
            writer.write(f)
        split_files.append(split_filename)

    return split_files

def send_to_upstage(pdf_path):
    url = "https://api.upstage.ai/v1/document-digitization"
    headers = {"Authorization": f"Bearer {api_key}"}
    files = {"document": open(pdf_path, "rb")}
    data = {
        "ocr": "force",
        "base64_encoding": "['table']",
        "model": "document-parse"
    }

    response = requests.post(url, headers=headers, files=files, data=data)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error for {pdf_path}: {response.status_code}, {response.text}")
        return None


In [38]:
api_key = "up_jj82424jbVXiORp7IphLE4DBphRUf"

split_files = split_pdf("./raw_pdfs/조사료-단면.pdf", "./split_pdfs", chunk_size=30)

for f in split_files:
    print(f"Sending {f} to Upstage...")
    result = send_to_upstage(f)
    if result:
        # JSON 결과 저장
        json_name = f.replace(".pdf", ".json")
        with open(json_name, "w", encoding="utf-8") as jf:
            json.dump(result, jf, ensure_ascii=False, indent=2)
        print(f"Saved result to {json_name}")

Sending ./split_pdfs/조사료-단면_1_30.pdf to Upstage...
Saved result to ./split_pdfs/조사료-단면_1_30.json
Sending ./split_pdfs/조사료-단면_31_60.pdf to Upstage...
Saved result to ./split_pdfs/조사료-단면_31_60.json
Sending ./split_pdfs/조사료-단면_61_90.pdf to Upstage...
Saved result to ./split_pdfs/조사료-단면_61_90.json
Sending ./split_pdfs/조사료-단면_91_120.pdf to Upstage...
Saved result to ./split_pdfs/조사료-단면_91_120.json
Sending ./split_pdfs/조사료-단면_121_150.pdf to Upstage...
Saved result to ./split_pdfs/조사료-단면_121_150.json
Sending ./split_pdfs/조사료-단면_151_180.pdf to Upstage...
Saved result to ./split_pdfs/조사료-단면_151_180.json
Sending ./split_pdfs/조사료-단면_181_210.pdf to Upstage...
Saved result to ./split_pdfs/조사료-단면_181_210.json
Sending ./split_pdfs/조사료-단면_211_240.pdf to Upstage...
Saved result to ./split_pdfs/조사료-단면_211_240.json
Sending ./split_pdfs/조사료-단면_241_270.pdf to Upstage...
Saved result to ./split_pdfs/조사료-단면_241_270.json
Sending ./split_pdfs/조사료-단면_271_300.pdf to Upstage...
Saved result to ./split_pdfs/조사료-단면

In [35]:
split_files

['./split_pdfs/농업기술길잡이127_부추_1_30.pdf',
 './split_pdfs/농업기술길잡이127_부추_31_60.pdf',
 './split_pdfs/농업기술길잡이127_부추_61_90.pdf',
 './split_pdfs/농업기술길잡이127_부추_91_120.pdf',
 './split_pdfs/농업기술길잡이127_부추_121_150.pdf',
 './split_pdfs/농업기술길잡이127_부추_151_180.pdf',
 './split_pdfs/농업기술길잡이127_부추_181_200.pdf']